# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from Config import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,makueni boma,-1.8041,37.6203,18.86,79,64,3.07,KE,1722284583
1,1,warrenton,-28.1140,24.8475,5.08,26,1,1.49,ZA,1722284584
2,2,fort mcmurray,56.7268,-111.3810,28.77,42,75,5.66,CA,1722284586
3,3,grytviken,-54.2811,-36.5092,-1.45,79,100,2.87,GS,1722284587
4,4,adamstown,-25.0660,-130.1015,21.73,81,40,7.36,PN,1722284588


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
                                    & (city_data_df["Wind Speed"] < 4.5) \
                                    & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,140,oulad teima,30.3947,-9.2090,21.60,75,0,4.24,MA,1722284755
156,156,alexandria,31.2156,29.9553,26.30,68,0,4.21,EG,1722284775
167,167,san sebastian de la gomera,28.0916,-17.1133,24.10,88,0,2.06,ES,1722284789
215,215,altud,43.7219,43.8658,22.25,45,0,0.68,RU,1722284851
267,267,constantia,44.1833,28.6500,23.38,70,0,2.88,RO,1722284915
276,276,misratah,32.3754,15.0925,26.85,62,0,3.59,LY,1722284925
295,295,sao vicente,-23.9631,-46.3919,22.93,66,0,1.34,BR,1722284950
307,307,madre de deus,-12.7408,-38.6208,26.03,77,0,4.24,BR,1722284964
393,393,keskin,39.6731,33.6136,21.40,29,0,2.51,TR,1722285078
395,395,saint-francois,46.4154,3.9054,24.80,74,0,2.06,FR,1722285080


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df["Hotel Name"] = ""

# Display sample data
hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
140,oulad teima,MA,30.3947,-9.2090,75,
156,alexandria,EG,31.2156,29.9553,68,
167,san sebastian de la gomera,ES,28.0916,-17.1133,88,
215,altud,RU,43.7219,43.8658,45,
267,constantia,RO,44.1833,28.6500,70,
276,misratah,LY,32.3754,15.0925,62,
295,sao vicente,BR,-23.9631,-46.3919,66,
307,madre de deus,BR,-12.7408,-38.6208,77,
393,keskin,TR,39.6731,33.6136,29,
395,saint-francois,FR,46.4154,3.9054,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "Config": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
oulad teima - nearest hotel: No hotel found
alexandria - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
altud - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
misratah - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
madre de deus - nearest hotel: No hotel found
keskin - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
cournonterral - nearest hotel: No hotel found
horsham - nearest hotel: No hotel found
al hamra' - nearest hotel: No hotel found
dazhong - nearest hotel: No hotel found
waat - nearest hotel: No hotel found
nguruka - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
140,oulad teima,MA,30.3947,-9.2090,75,No hotel found
156,alexandria,EG,31.2156,29.9553,68,No hotel found
167,san sebastian de la gomera,ES,28.0916,-17.1133,88,No hotel found
215,altud,RU,43.7219,43.8658,45,No hotel found
267,constantia,RO,44.1833,28.6500,70,No hotel found
276,misratah,LY,32.3754,15.0925,62,No hotel found
295,sao vicente,BR,-23.9631,-46.3919,66,No hotel found
307,madre de deus,BR,-12.7408,-38.6208,77,No hotel found
393,keskin,TR,39.6731,33.6136,29,No hotel found
395,saint-francois,FR,46.4154,3.9054,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)